In [29]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.formula.api import glm

from sklearn.metrics import mean_absolute_error,mean_squared_error, accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#### correlated feature removed 

In [30]:
df=pd.read_csv(r'C:\Melbourne_Escooter\RealDataset\FinalResults\SA1\Features-trips\Final\final version\AllFeatures-original2.csv')

In [31]:
df.fillna(0, inplace=True)

In [32]:
column=['tripDensity','House perc','entropy','15-29 %','above65 %','female%','car owner %','male%','with no u 15 children sa1','with u 15 children sa1','no_childern Sa1']
df_depe=df.drop(column, axis=1)
df_depe

,SA1_CODE21,date,hour,trainDensity,busDensity,tramDensity,Cafe perc,Office perc,Shops perc,mxi,recreationCount,campusCount,5-14 %,30-39 %,40-49 %,50-64 %,Population density 2021,rainfall,min_temp
0,20604111702,1,0,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,19.330746,7.422579,8.152377,9456.2,0,7.4
1,20604111702,1,1,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,19.330746,7.422579,8.152377,9456.2,0,7.4
2,20604111702,1,2,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,19.330746,7.422579,8.152377,9456.2,0,7.4
3,20604111702,1,3,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,19.330746,7.422579,8.152377,9456.2,0,7.4
4,20604111702,1,4,0.0,38.684720,19.34236,0.576336,0.022901,0.030534,12.977099,0,0,3.536138,19.330746,7.422579,8.152377,9456.2,0,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56779,20604151016,7,19,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,34.053583,13.562667,0.000000,9238.5,1,8.0
56780,20604151016,7,20,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,34.053583,13.562667,0.000000,9238.5,1,8.0
56781,20604151016,7,21,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,34.053583,13.562667,0.000000,9238.5,1,8.0
56782,20604151016,7,22,0.0,46.728972,0.00000,0.803571,0.071429,0.035714,41.071429,0,0,1.813717,34.053583,13.562667,0.000000,9238.5,1,8.0


In [33]:
df_depe=df_depe.drop(['SA1_CODE21'], axis=1)

In [34]:
df_depe["hour"].replace({0:1,1:1,2:1,3:2,4:2,5:2,6:3,7:3,8:3,9:4,10:4,11:4,12:5,13:5,14:5,15:6,16:6,17:6,18:7,19:7,20:7,21:8,22:8,23:8}, inplace=True)
df_depe["date"].replace({1:1,2: 1, 3: 1,4: 1,5: 1,6: 2,7: 2}, inplace=True)
df_depe['trainDensity'] = df['trainDensity'].map(lambda x: 1 if x>0 else 0)

In [35]:
categorical =['hour','date','rainfall','trainDensity']
for col in categorical:
    df_depe[col]=df_depe[col].astype('category')

In [36]:
df_depe=pd.get_dummies(df_depe, drop_first=True)


In [37]:
df_depe.columns

Index(['busDensity', 'tramDensity', 'Cafe perc', 'Office perc', 'Shops perc',
       'mxi', 'recreationCount', 'campusCount', '5-14 %', '30-39 %', '40-49 %',
       '50-64 %', 'Population density 2021', 'min_temp', 'date_2', 'hour_2',
       'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8',
       'trainDensity_1', 'rainfall_1'],
      dtype='object')

In [38]:
scale= StandardScaler()
df_sc= scale.fit_transform(df_depe)
df_sc=pd.DataFrame(df_sc, columns=df_depe.columns)

In [39]:
df['tripDensity'] = df['tripDensity'].map(lambda x: 1 if x>49.0 else 0)
df['tripDensity']=df['tripDensity'].astype('category')

In [40]:
y=df['tripDensity']
X=df_sc

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [55]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1,random_state=42,max_iter=1000)

In [56]:
model = clf.fit(X_train, y_train)

In [44]:
ypredict = clf.predict(X_test)

In [45]:
print("MAE %f" %mean_absolute_error(y_test, ypredict))


MAE 0.197059


In [46]:
print("MSE  %f" %mean_squared_error(y_test, ypredict))

MSE  0.197059


In [47]:
import numpy as np
print("RMSE  %f" %np.sqrt(mean_squared_error(y_test, ypredict)))

RMSE  0.443913


In [48]:
f1_score(y_test, ypredict)

0.2428958051420839

In [49]:
y_pred_proba = clf.predict_proba(X_test)[::,1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.7758328461054573


In [50]:
import statsmodels.api as sm
X_train =sm.add_constant(X_train)

In [51]:
model=sm.Logit(y_train,X_train)

In [52]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.415796
         Iterations 7


In [53]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:            tripDensity   No. Observations:                45427
Model:                          Logit   Df Residuals:                    45402
Method:                           MLE   Df Model:                           24
Date:                Sat, 24 Sep 2022   Pseudo R-squ.:                  0.1614
Time:                        16:14:17   Log-Likelihood:                -18888.
converged:                       True   LL-Null:                       -22523.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -1.7993      0.016   -110.446      0.000      -1.831      -1.767
busDensity                 -0.1458      0.014    -10.318      0.000      -0.173      -0.118
tramDensity                 0.2196      0.015     15.023      0.000       0.191       0.248
Cafe perc                   0.4131      0.016     26.133      0.000       0.382       0.444
Office perc                 0.1330      0.011     12.013      0.000       0.111       0.155
Shops perc                  0.1542      0.011     13.431      0.000       0.132       0.177
mxi                         0.2661      0.019     13.979      0.000       0.229       0.303
recreationCount            -0.2826      0.029     -9.831      0.000      -0.339      -0.226
campusCount                -0.0560      0.013     -4.380      0.000      -0.081      -0.031
5-14 %                     -0.2369      0.023    -10.340      0.000      -0.282      -0.192
30-39 %                     0.3681      0.021     17.880      0.000       0.328       0.408
40-49 %                    -0.3941      0.031    -12.726      0.000      -0.455      -0.333
50-64 %                    -0.0213      0.021     -1.023      0.306      -0.062       0.020
Population density 2021     0.1083      0.019      5.612      0.000       0.070       0.146
min_temp                    0.1703      0.018      9.407      0.000       0.135       0.206
date_2                      0.3835      0.019     19.719      0.000       0.345       0.422
hour_2                     -0.2909      0.020    -14.313      0.000      -0.331      -0.251
hour_3                     -0.1682      0.019     -8.906      0.000      -0.205      -0.131
hour_4                      0.0170      0.017      0.981      0.327      -0.017       0.051
hour_5                      0.1295      0.017      7.724      0.000       0.097       0.162
hour_6                      0.2282      0.016     13.928      0.000       0.196       0.260
hour_7                      0.1798      0.017     10.831      0.000       0.147       0.212
hour_8                      0.1801      0.016     10.918      0.000       0.148       0.212
trainDensity_1             -0.0748      0.015     -5.040      0.000      -0.104      -0.046
rainfall_1                 -0.1312      0.015     -8.518      0.000      -0.161      -0.101
===========================================================================================
"""

In [ ]:
# cross validation

In [57]:
test_score= cross_val_score(model, X_test, y_test, scoring='neg_mean_squared_error')

In [60]:
cv_mse=np.mean(np.absolute(test_score))

In [61]:
round(np.sqrt(cv_mse),2)

0.44